<a href="https://colab.research.google.com/github/YinmiAlas/DS-Unit-2-Linear-Models/blob/master/LS_DS_213_Ridge_Regression_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [171]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [193]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [194]:
df.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019


In [195]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)
df.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019


In [196]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [197]:
df.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019


In [198]:
# subsetting data
df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df[df['SALE_PRICE'] >= 100000]
df = df[df['SALE_PRICE'] <= 1000000]
df.head(1)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019


In [199]:
#splitting data
# adding datime index column to for a esay split
df['SALE_DATE'] = df['SALE_DATE'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df = df.set_index(df['SALE_DATE'])
df = df.sort_index()

sale_date_train = df['2019-01-01':'2019-03-31']

sale_date_test = df['2019-04-01':'2019-04-30']

sale_date_train.shape, sale_date_test.shape

((2316, 21), (590, 21))

In [202]:
# x matrices and y vectors
target = 'SALE_PRICE'
               
features = ['TOTAL_UNITS',	'LAND_SQUARE_FEET',	'GROSS_SQUARE_FEET',	'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE']

sale_date_train_x = sale_date_train[features]

sale_date_train_y = sale_date_train[target]

sale_date_test_x = sale_date_test[features]

sale_date_test_y = sale_date_test[target]

sale_date_train_x.shape, sale_date_test_y.shape

((2316, 5), (590,))

In [203]:
# one-hot-encoding
import category_encoders as ce 

categorical_encoder = ce.OneHotEncoder(use_cat_names=True)

sale_date_train_x = categorical_encoder.fit_transform(sale_date_train_x)
sale_date_test_x = categorical_encoder.transform(sale_date_test_x)

sale_date_train_x.head(2)

,TOTAL_UNITS,"LAND_SQUARE_FEET_6,800","LAND_SQUARE_FEET_4,000","LAND_SQUARE_FEET_3,500","LAND_SQUARE_FEET_1,710","LAND_SQUARE_FEET_2,000","LAND_SQUARE_FEET_3,000","LAND_SQUARE_FEET_1,800","LAND_SQUARE_FEET_5,000","LAND_SQUARE_FEET_2,400","LAND_SQUARE_FEET_3,700","LAND_SQUARE_FEET_2,500","LAND_SQUARE_FEET_2,435",LAND_SQUARE_FEET_760,"LAND_SQUARE_FEET_3,920","LAND_SQUARE_FEET_2,626","LAND_SQUARE_FEET_1,383","LAND_SQUARE_FEET_2,200","LAND_SQUARE_FEET_3,570","LAND_SQUARE_FEET_8,600","LAND_SQUARE_FEET_7,107","LAND_SQUARE_FEET_4,629","LAND_SQUARE_FEET_3,800","LAND_SQUARE_FEET_5,500","LAND_SQUARE_FEET_9,383","LAND_SQUARE_FEET_3,984","LAND_SQUARE_FEET_4,900","LAND_SQUARE_FEET_1,635","LAND_SQUARE_FEET_2,160","LAND_SQUARE_FEET_3,068","LAND_SQUARE_FEET_4,482","LAND_SQUARE_FEET_3,655","LAND_SQUARE_FEET_1,403","LAND_SQUARE_FEET_4,410","LAND_SQUARE_FEET_2,325","LAND_SQUARE_FEET_2,527","LAND_SQUARE_FEET_2,058","LAND_SQUARE_FEET_3,228","LAND_SQUARE_FEET_2,217","LAND_SQUARE_FEET_1,917",...,LAND_SQUARE_FEET_864,"LAND_SQUARE_FEET_4,378","LAND_SQUARE_FEET_5,250","LAND_SQUARE_FEET_2,052","LAND_SQUARE_FEET_1,470","LAND_SQUARE_FEET_3,071","LAND_SQUARE_FEET_2,270","LAND_SQUARE_FEET_5,425","LAND_SQUARE_FEET_1,767","LAND_SQUARE_FEET_1,293","LAND_SQUARE_FEET_2,233","LAND_SQUARE_FEET_5,005","LAND_SQUARE_FEET_1,235","LAND_SQUARE_FEET_1,060",LAND_SQUARE_FEET_946,"LAND_SQUARE_FEET_1,698","LAND_SQUARE_FEET_3,990","LAND_SQUARE_FEET_3,012","LAND_SQUARE_FEET_2,231","LAND_SQUARE_FEET_4,485","LAND_SQUARE_FEET_1,918","LAND_SQUARE_FEET_3,338","LAND_SQUARE_FEET_2,755","LAND_SQUARE_FEET_3,230","LAND_SQUARE_FEET_2,121","LAND_SQUARE_FEET_6,900","LAND_SQUARE_FEET_4,361","LAND_SQUARE_FEET_3,910",GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_A3
SALE_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1325.0,1930.0,1,0,0,0,0,0,0,0,0,0
2019-01-01,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2001.0,1940.0,0,1,0,0,0,0,0,0,0,0


In [229]:
#using selecting features with SelectKBest 
from sklearn.feature_selection import SelectKBest, f_regression

selecting_features = SelectKBest(k=10)

X_train_selected = selecting_features.fit_transform(sale_date_train_x, sale_date_train_y)
X_test_selected = selecting_features.transform(sale_date_test_x)

X_train_selected.shape, X_test_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


((2316, 10), (590, 10))

In [224]:
from sklearn.linear_model import Ridge
#alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
ridge = Ridge(normalize=True)
ridge.fit(X_train_selected, sale_date_train_y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [225]:
from sklearn.metrics import mean_absolute_error

y_pred = ridge.predict(X_test_selected)
mae = mean_absolute_error(sale_date_test_y, y_pred)
mae
print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

Test Mean Absolute Error: $157,732 



In [230]:
for k in range(1, len(sale_date_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selecting_features.fit_transform(sale_date_train_x, sale_date_train_y)
    X_test_selected = selecting_features.transform(sale_date_test_x)

    ridge = Ridge(normalize=True)
    ridge.fit(X_train_selected, sale_date_train_y)
    y_pred = ridge.predict(X_test_selected)
    mae = mean_absolute_error(sale_date_test_y, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $157,556 

2 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


Test Mean Absolute Error: $157,556 

3 features
Test Mean Absolute Error: $157,556 

4 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


Test Mean Absolute Error: $157,556 

5 features
Test Mean Absolute Error: $157,556 

6 features


...

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


Test Mean Absolute Error: $157,556 

19 features
Test Mean Absolute Error: $157,556 

20 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


Test Mean Absolute Error: $157,556 

21 features
Test Mean Absolute Error: $157,556 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


#Hey TL sorry but i will need your help in this part i will stop my assigment here can you help me figuring out what i got this result? 
#Thanks.